In [13]:
!pip install streamlit opencv-python-headless numpy requests beautifulsoup4 matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 979.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.7 MB/s eta 0:00:00


In [66]:
%%writefile app.py
import time
import cv2
import numpy as np

vehicle_classes = {"car", "bus", "motorcycle", "truck"}


# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def detect_objects(image):
    """Detect objects using the YOLO model."""
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    return net.forward(output_layers)

def process_detections(detections, width, height):
    """Process YOLO detections and filter vehicles."""
    vehicle_classes = {"car", "bus", "motorcycle", "truck"}
    boxes, confidences, class_ids = [], [], []

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                x = int(detection[0] * width)
                y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return indexes, boxes, class_ids


import requests
from bs4 import BeautifulSoup
import cv2
import numpy as np

def fetch_image(city):
    """Fetch an image from the traffic camera."""
    url = f"https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/{city}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    img_src = [img['src'] for img in soup.find_all('img') if 'datamall' in img['src']]
    if not img_src:
        return None
    img = []
    for i in range(3):
      img_url = "https:" + img_src[i]
      img_response = requests.get(img_url)
      img_array = np.frombuffer(img_response.content, np.uint8)
      imgcode = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
      img.append(imgcode)
    return img


def Draw_Predictions(image, indexes, boxes, class_ids):
    """Draw bounding boxes, labels, and print the count of each vehicle type."""
    font = cv2.FONT_HERSHEY_PLAIN
    vehicle_counts = {}
    if len(indexes) > 0:
      for i in indexes.flatten():
          x, y, w, h = boxes[i]
          label = str(classes[class_ids[i]])

          if label in vehicle_classes:
              # Increment the count of the detected vehicle type
              vehicle_counts[label] = vehicle_counts.get(label, 0) + 1

              color = (0, 255, 0)  # Green for vehicles
              cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
              cv2.putText(image, label, (x, y - 10), font, 2, color, 2)

    # Convert BGR to RGB for display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb,vehicle_counts

import streamlit as st
import matplotlib.pyplot as plt
import random

# Import your functions from other modules
#rom traffic_image import fetch_image
#from yolo_model import detect_objects, process_detections

st.set_page_config(page_title="Live Traffic Monitoring", layout="wide")
st.title("Live Traffic Monitoring System")
# List of cities and angles
city_options = {
    "Woodlands Checkpoint": "woodlands",
    "Seletar Expressway (SLE)": "sle",
    "Tampines Expressway (TPE)": "tpe",
    "Bukit Timah Expressway (BKE)": "bke",
    "Kallang-Paya Lebar Expressway (KPE)": "kpe",
    "Central Expressway (CTE)": "cte",
    "Pan-Island Expressway (PIE)": "pie",
    "East Coast Parkway (ECP)": "ecp",
    "Ayer Rajah Expressway (AYE)": "aye",
    "Kranji Expressway (KJE)": "kje"
}

# Dropdown with display names, returning backend value
city = st.selectbox("Select a city:", options=list(city_options.keys()))

# Get the backend value
backend_value = city_options[city]

st.write(f"You selected: {city}")
#cities = ['woodlands', 'sle', 'tpe', 'bke', 'kpe', 'cte', 'pie', 'ecp', 'aye', 'kje']
angles = ['Angle 1', 'Angle 2', 'Angle 3']

#city = st.selectbox("Select a city:", cities)
placeholder = st.empty()  # Reserve space for dynamic updates
with placeholder.container():
      if st.button("Fetch and Analyze Traffic Images"):
          st.header(f"Traffic Monitoring for {city.title()}")
          img = fetch_image(backend_value)  # Fetch image for the given city
          if img is not None:
              cols = st.columns(3)  # Create three columns for three angles

              for idx, angle in enumerate(angles):
                  with cols[idx]:  # Load each image in a separate column
                      st.subheader(angle)
                      height, width, _ = img[idx].shape
                      detections = detect_objects(img[idx])
                      indexes, boxes, class_ids = process_detections(detections, width, height)
                      image_rgb,vehicle_counts = Draw_Predictions(img[idx], indexes, boxes, class_ids)


                      # Display the image
                      st.image(image_rgb, channels="BGR", use_column_width=True)


                      # Display vehicle counts as a chart
                      if vehicle_counts:
                          fig, ax = plt.subplots()
                          ax.bar(vehicle_counts.keys(), vehicle_counts.values(), color='skyblue')
                          ax.set_title(f"Vehicle Counts for {angle}")
                          ax.set_xlabel("Vehicle Type")
                          ax.set_ylabel("Count")
                          st.pyplot(fig)
                      else:
                          st.write("No vehicles detected.")


Overwriting app.py


In [67]:
!wget -q -O - ipv4.icanhazip.com

34.23.184.171


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.184.171:8501

your url is: https://giant-clowns-enjoy.loca.lt
